In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from transformers import *


We will use weights and biases for tracking experiments and runs. Project page : https://wandb.ai/tasmiah-tahsin/fake-news-blurr

# Load dataset

In [ ]:
# fake = pd.read_csv("/content/drive/MyDrive/datasets/Fake-1K.csv")
# fake_syn = pd.read_csv("/content/drive/MyDrive/datasets/fake_synthetic.csv")
# authentic = pd.read_csv("/content/drive/MyDrive/datasets/Authentic-48K.csv",engine='python',error_bad_lines=False,warn_bad_lines=True,nrows=15000)
# df = pd.concat([authentic[['headline','content','label']],fake[['headline','content','label']],fake_syn[['headline','content','label']]])
# df.reset_index(drop=True,inplace=True)

In [ ]:
split1 = pd.read_csv("/content/drive/MyDrive/datasets/work2.csv")
split1.shape

(1200, 2)

##import tokenizer 

In [ ]:
from transformers import BertTokenizer, AutoTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

Loading BERT tokenizer...


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout"

##Code for Alt line Truncation

In [ ]:
def calc_len_wtokenizer(sent):
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    length = len(input_ids)
    return length

In [ ]:
def calc_len(sent):
    word_count = len(sent.split())
    return word_count

In [ ]:
def trunc_alt_sent(str_w):
    x = str_w.rsplit("। ")
    n = len(x)
    if n==1:
      x = str_w.rsplit("।")
      n = len(x)
    res_str = "default: "
    l2 = []
    for i in range(0,n):
        if(i%2==0):
            #print(x[i])
            l2.append(x[i])

    res_str = "। ".join(l2)

    #print(res_str)
    return res_str
    

In [ ]:
import time
#timeout = time.time() + 60   # 1 minutes from now

In [ ]:
# timeout = time.time() + 30
# while True:    
#     if time.time() > timeout:
#       print('timeout')
#       break

In [ ]:
a_string = split1.iloc[0]['text']
l = calc_len_wtokenizer(a_string)
print(l)

450


In [ ]:
itr=0
sum_count=0

# For every sentence...
for sent in split1['text']:   
    
    timeout = time.time() + 60*2
    while(1):
        if(calc_len_wtokenizer(sent)<512):
            break
        
        sum_count+=1
        
        if(time.time() > timeout):
            print('timeout. eror in row: ', itr)
            break
        sent = trunc_alt_sent(sent)
    
    split1.at[itr,'text'] = sent
    
    if(itr%2500==0):
      print(itr)
    itr+=1
print("successfully done. sum count: ", sum_count)

0
successfully done. sum count:  0


In [ ]:
split1.to_csv('/content/drive/MyDrive/datasets/summarized_test.csv', index=False)

In [ ]:
split1.shape

(10016, 2)

##custom truncation 

In [ ]:
itr = 903
a_string = split1.iloc[itr]['text']
a_string

'ফেসবুক কি বাংলাদেশিদের বাকস্বাধীনতা হরণ করছে?ফেসবুক বিভিন্নভাবে বাংলাদেশের মানুষের বাকস্বাধীনতা হরণের চেষ্টা করছে বলে অভিযোগ করেছেন একাধিক সাংবাদিক, অ্যাক্টিভিস্ট। সুনির্দিষ্ট সংখ্যা জানা না গেলেও বাংলাদেশে ২০ হাজারের বেশি ফেসবুক অনুসারী আছেন, এমন বেশ কয়েকজন অ্যাক্টিভিস্ট, সাংবাদিক এবং রাজনীতিবিদের এমন অভিজ্ঞতা রয়েছে। আর তারপর ফেসবুক সেই বিষয়ে ব্যবস্থা গ্রহণ করে কিংবা নীতি ভঙ্গ হয়নি মনে করলে কোনো ব্যবস্থা নেয় না। জার্মানির হামবুর্গ শহরে অবস্থিত প্রতিষ্ঠানটির কার্যালয় থেকে ক্লাউস গোর্নি ইমেলে জানান, বাংলাসহ বিশ্বের ৫০টির বেশি ভাষায় প্রকাশিত কন্টেন্ট মনিটর করতে ২৪ ঘণ্টাই কাজ করছে ফেসবুকের ‘কন্টেন্ট রিভিউ টিম’। গোর্নি জানান, অন্যের নামে ফেক অ্যাকাউন্ট খোলার বিষয়টি ফেসবুক একেবারেই সহ্য করে না এবং নানা পন্থা এ ধরনের অ্যাকাউন্ট শনাক্তে সহায়তা করে এবং সেগুলো বন্ধ করে দেয়'

In [ ]:
# x = a_string.rsplit("!!")
# n = len(x)
# # for i in range(0,n):
# #     if(i%2==0):
# #         print(n,x[i])
# x[0]
# split1.at[itr,'text'] = x[0]

In [ ]:
def trunc_custom(str_w):
    x = str_w.rsplit("। ")
    n = len(x)
    res_str = "default: "
    l2 = []
    for i in range(0,n):
        if(i%2==0):
            #print(x[i])
            l2.append(x[i])

    res_str = "। ".join(l2)

    #print(res_str)
    return res_str
    

In [ ]:
#itr = 4435
sent = split1.iloc[itr]['text']
timeout = time.time() + 30
while(1):
    if(calc_len_wtokenizer(sent)<512):
        break
  
    
    if(time.time() > timeout):
        print('timeout. eror in row: ', itr)
        break
    sent = trunc_custom(sent)

split1.at[itr,'text'] = sent

print("successfully done, length: ", calc_len_wtokenizer(sent))

successfully done, length:  316


In [ ]:
# split1.to_csv('/content/drive/MyDrive/datasets/work3.csv', index=False)

##Code for Slicing df before summarization


In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--csebuetnlp--mT5_multilingual_XLSum/snapshots/2437a524effdbadc327ced84595508f1e32025b3/config.json
Model config MT5Config {
  "_name_or_path": "csebuetnlp/mT5_multilingual_XLSum",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "length_penalty": 0.6,
  "max_length": 84,
  "model_type": "mt5",
  "no_repeat_ngram_size": 2,
  "num_beams": 4,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tok

In [ ]:
counter = 0
i=0

# For every sentence...
for sent in final_df['text']:   

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Only transfering greater than 500 token_length
    length = len(input_ids)
    if(length>500):
        sel_df = sel_df.append(final_df.iloc[i]) 
        final_df = final_df.drop(i)
        counter+=1    
    
    i+=1              

print('number of sum text: ', counter)

IndexError: ignored

In [ ]:
sel_df.shape

(3931, 3)

In [ ]:
final_df.shape

(6085, 3)

In [ ]:
sel_df.to_csv('/content/drive/MyDrive/datasets/selected_forsum.csv')
final_df.to_csv('/content/drive/MyDrive/datasets/mod_final.csv')

##slicing into 9 parts

In [ ]:
split1 = sel_df.iloc[:440, 0:]
split2 = sel_df.iloc[440:880, 0:]
split3 = sel_df.iloc[880:1320, 0:]
split4 = sel_df.iloc[1320:1760, 0:]
split5 = sel_df.iloc[1760:2200, 0:]
split6 = sel_df.iloc[2200:2640, 0:]
split7 = sel_df.iloc[2640:3080, 0:]
split8 = sel_df.iloc[3080:3520, 0:]
split9 = sel_df.iloc[3520:3930, 0:]

In [ ]:
split9.shape, split8.shape

((410, 3), (440, 3))

In [ ]:
split1.to_csv('split1.csv')
split2.to_csv('split2.csv')
split3.to_csv('split3.csv')
split4.to_csv('split3.csv')
split5.to_csv('split5.csv')
split6.to_csv('split6.csv')
split7.to_csv('split7.csv')
split8.to_csv('split8.csv')
split9.to_csv('split9.csv')


In [ ]:
split1.to_csv('/content/drive/MyDrive/datasets/split1.csv')

In [ ]:
split2.to_csv('/content/drive/MyDrive/datasets/split2.csv')
split3.to_csv('/content/drive/MyDrive/datasets/split3.csv')
split4.to_csv('/content/drive/MyDrive/datasets/split4.csv')
split5.to_csv('/content/drive/MyDrive/datasets/split5.csv')
split6.to_csv('/content/drive/MyDrive/datasets/split6.csv')
split7.to_csv('/content/drive/MyDrive/datasets/split7.csv')
split8.to_csv('/content/drive/MyDrive/datasets/split8.csv')
split9.to_csv('/content/drive/MyDrive/datasets/split9.csv')

##Saving the csv

In [ ]:
final_df.to_csv('sum_train.csv', index=False)
path_tsum = '/content/drive/MyDrive/datasets/sum_train.csv'
with open(path1, 'w') as f:
  final_df.to_csv(f, index=False)

# Resources

* fastai paper : https://arxiv.org/pdf/2002.04688.pdf
* [BERT Fine-Tuning Tutorial with PyTorch · Chris McCormick](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)
* [multilingual bert](https://huggingface.co/bert-base-multilingual-cased)
* https://github.com/cdpierse/transformers-interpret
* https://blog.dataiku.com/the-learning-rate-finder-technique-how-reliable-is-it